### Download Acustic Emission Dataset

In [4]:
import gdown
## While creating link, select anyone with the link, and additionally, uc?id= should be added like in below.
url = 'https://drive.google.com/uc?id=1LIkyqBeKWk5GHyiuVL-lUBA-JD7g5omA'
output = 'matlab15000.mat'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1LIkyqBeKWk5GHyiuVL-lUBA-JD7g5omA
To: /kaggle/working/matlab15000.mat
100%|██████████| 105M/105M [00:00<00:00, 136MB/s]  


'matlab15000.mat'

### Use GPU and import necessary libraries

In [5]:
import tensorflow as tf
from tensorflow import keras

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Veysi ADIN 19 SEPT 2022
from pandas import read_csv
from numpy import set_printoptions
from sklearn import datasets, linear_model
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import random
# Using GPU
import os
import scipy.io as scpy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'  # Set to -1 if CPU should be used CPU = -1 , GPU = 0

gpus = tf.config.experimental.list_physical_devices('GPU')
cpus = tf.config.experimental.list_physical_devices('CPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
elif cpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        logical_cpus= tf.config.experimental.list_logical_devices('CPU')
        print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPU")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


1 Physical GPUs, 1 Logical GPUs


### Login wandb interface with API key

In [7]:
import wandb
from wandb.keras import WandbCallback

wandb.login(key='0eeb933c285c4fbafee9f8d0f3d524022593e894')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Load dataset and split it 80,10,10 for train,validation and test

In [8]:
data = scpy.loadmat("matlab15000.mat")
# Extracting x_train from the mat file dictionary.
x_data = data["XTrain"]
# Extracting y_train from the mat file dictionary.
y_data = data["unnamed"]
# Converting x_train and y_train to a numpy array.
x_data = np.array(x_data,dtype='float32')
y_train = np.array(y_data,dtype='float32')
# Verifying the shapes.
print(x_data.shape)
print(y_data.shape)

SEED = 99
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
# split into train test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=SEED)


x_val = x_test[:1500]
y_val = y_test[:1500]
x_test_to_use = x_test[1500:]
y_test_to_use = y_test[1500:]
print(f" {len(x_train), len(x_val), len(x_test_to_use)}")

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
BATCH_SIZE = 64
# SHUFFLE_BUFFER_SIZE = 100
train_dataset = train_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

(15000, 1000)
(15000, 1)
 (12000, 1500, 1500)


In [16]:
def train_step(x, y, model, optimizer, loss_fn, train_acc_metric):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)

    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y, logits)

    return loss_value

In [17]:
def test_step(x, y, model, loss_fn, val_acc_metric):
    val_logits = model(x, training=False)
    loss_value = loss_fn(y, val_logits)
    val_acc_metric.update_state(y, val_logits)

    return loss_value

In [18]:
def train(train_dataset, val_dataset,  model, optimizer,
          train_acc_metric, val_acc_metric,
          epochs=10,  log_step=200, val_log_step=50):
  
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []   
        val_loss = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            loss_value = train_step(x_batch_train, y_batch_train, 
                                    model, optimizer, 
                                    loss_fn, train_acc_metric)
            train_loss.append(float(loss_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
            val_loss_value = test_step(x_batch_val, y_batch_val, 
                                       model, loss_fn, 
                                       val_acc_metric)
            val_loss.append(float(val_loss_value))
            
        # Display metrics at the end of each epoch
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))

        # Reset metrics at the end of each epoch
        train_acc_metric.reset_states()
        val_acc_metric.reset_states()

        # ⭐: log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss),
                   'acc': float(train_acc), 
                   'val_loss': np.mean(val_loss),
                   'val_acc':float(val_acc)})

In [ ]:
def make_conv_model():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Reshape((1000,1,1),name='Reshape')(inputs)
    x2 = tf.keras.layers.Conv2D(8,5,padding='same')(x1)
    x3= tf.keras.layers.Activation('relu')(x2)
    x4 = tf.keras.layers.MaxPooling2D(2,2, padding='same')(x3)
    x5 = tf.keras.layers.Flatten()(x4)
    outputs = tf.keras.layers.Dense(3,activation='softmax', name="output")(x5)
    return keras.Model(inputs=inputs, outputs=outputs)
def make_conv_model_paper():
    inputs=keras.Input(shape=(x_train.shape[1],))
    x0 = tf.keras.layers.Reshape((1000,1),name='Reshape')(inputs)
    x1=tf.keras.layers.Conv1D(32,7)(x0)
    x2=tf.keras.layers.BatchNormalization()(x1)
    x3=tf.keras.layers.Activation('relu')(x2)
    x4=tf.keras.layers.MaxPooling1D(pool_size=8)(x3)
    x5=tf.keras.layers.Conv1D(48,5)(x4)
    x6=tf.keras.layers.BatchNormalization()(x5)
    x7=tf.keras.layers.Activation('relu')(x6)
    x8=tf.keras.layers.MaxPooling1D(pool_size=4)(x7)
    x9=tf.keras.layers.MaxPooling1D(pool_size=4)(x8)
    x10=tf.keras.layers.Dropout(0.3)(x9)
    x11=tf.keras.layers.Flatten()(x10)
    x12=tf.keras.layers.Dense(32)(x11)
    x13=tf.keras.layers.Dense(32)(x12)
    outputs=tf.keras.layers.Dense(3, activation='softmax')(x13)
    return keras.Model(inputs=inputs, outputs=outputs)
def make_dense_model():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Flatten(input_shape=(1000,))(inputs)
    x2 = keras.layers.Dense(4, activation="relu")(x1)
    outputs = keras.layers.Dense(3, name="output")(x2)
    
    return keras.Model(inputs=inputs, outputs=outputs)
def make_dense_model_swish():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Flatten(input_shape=(1000,))(inputs)
    x2 = keras.layers.Dense(4, activation="swish")(x1)
    outputs = keras.layers.Dense(3, name="output")(x2)
    
    return keras.Model(inputs=inputs, outputs=outputs)

def make_dense_model_tanh():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Flatten(input_shape=(1000,))(inputs)
    x2 = keras.layers.Dense(4, activation="tanh")(x1)
    outputs = keras.layers.Dense(3, name="output")(x2)
    
    return keras.Model(inputs=inputs, outputs=outputs)
def make_dense_model_elu():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Flatten(input_shape=(1000,))(inputs)
    x2 = keras.layers.Dense(4, activation="elu")(x1)
    outputs = keras.layers.Dense(3, name="output")(x2)
    
    return keras.Model(inputs=inputs, outputs=outputs)
def make_dense_model_gelu():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Flatten(input_shape=(1000,))(inputs)
    x2 = keras.layers.Dense(4, activation="gelu")(x1)
    outputs = keras.layers.Dense(3, name="output")(x2)
    
    return keras.Model(inputs=inputs, outputs=outputs)
def make_dense_model_plus():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Flatten(input_shape=(1000,))(inputs)
    x2 = keras.layers.Dense(4, activation="softplus")(x1)
    outputs = keras.layers.Dense(3, name="output")(x2)
    
    return keras.Model(inputs=inputs, outputs=outputs)
def make_dense_model_hard_sigmoid():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Flatten(input_shape=(1000,))(inputs)
    x2 = keras.layers.Dense(4, activation="hard_sigmoid")(x1)
    outputs = keras.layers.Dense(3, name="output")(x2)
    
    return keras.Model(inputs=inputs, outputs=outputs)
def make_dense_model_sigmoid():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Flatten(input_shape=(1000,))(inputs)
    x2 = keras.layers.Dense(4, activation="sigmoid")(x1)
    outputs = keras.layers.Dense(3, name="output")(x2)
    
    return keras.Model(inputs=inputs, outputs=outputs)
def make_dense_act_sigmoid():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Flatten(input_shape=(1000,))(inputs)
    x2 = keras.layers.Dense(4, activation="elu")(x1)
    outputs = keras.layers.Dense(3, activation='sigmoid',name="output")(x2)
    
    return keras.Model(inputs=inputs, outputs=outputs)
def make_dense_act_softmax():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Flatten(input_shape=(1000,))(inputs)
    x2 = keras.layers.Dense(4, activation="sigmoid")(x1)
    outputs = keras.layers.Dense(3, activation='softmax',name="output")(x2)
    
    return keras.Model(inputs=inputs, outputs=outputs)

In [23]:
# initialize wandb with your project name and optionally with configutations.
# play around with the config values and see the result on your wandb dashboard.
config = {
              "learning_rate": 0.001,
              "epochs": 100,
              "batch_size": 64,
              "log_step": 200,
              "val_log_step": 50,
           }

run = wandb.init(project='dense-model-activations-fixed-seed', config=config,group='Activations',name='Act-Softmax-Logist-True')
config = wandb.config

# Initialize model.
model = make_dense_act_softmax()
model.summary()
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.RMSprop(learning_rate=config.learning_rate)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

train(train_dataset,
      test_dataset, 
      model,
      optimizer,
      train_acc_metric,
      val_acc_metric,
      epochs=config.epochs, 
      log_step=config.log_step, 
      val_log_step=config.val_log_step)

run.finish()  # In Jupyter/Colab, let us know you're finished!

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1000)]            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 4004      
_________________________________________________________________
output (Dense)               (None, 3)                 15        
Total params: 4,019
Trainable params: 4,019
Non-trainable params: 0
_________________________________________________________________

Start of epoch 0
Training acc over epoch: 0.7479
Validation acc: 0.8413

Start of epoch 1
Training acc over epoch: 0.8950
Validation acc: 0.9153

Start of epoch 2
Training acc over epoch: 0.9420
Validation acc: 0.9493

Start of epoch 3
Training acc over 

acc,▁▆▇█████████████████████████████████████
epochs,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▆██████████████████████████████████████
val_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂
acc,0.99917
epochs,99
loss,0.00746
val_acc,0.98133
val_loss,0.12146


In [ ]:
# initialize wandb with your project name and optionally with configutations.
# play around with the config values and see the result on your wandb dashboard.
config = {
              "learning_rate": 0.001,
              "epochs": 100,
              "batch_size": 64,
              "log_step": 200,
              "val_log_step": 50,
           }

run = wandb.init(project='dense-model-activations-fixed-seed', config=config,  group='RMSprop_L1e-3',name='Dense_Model_Sigmoid')
config = wandb.config

# Initialize model.
# make_dense_model_elu()
# make_dense_model_gelu
# make_dense_model_plus
# make_dense_model_hard_sigmoid
# make_dense_model_sigmoid

model = make_dense_model_sigmoid()
model.summary()
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.RMSprop(learning_rate=config.learning_rate)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

train(train_dataset,
      test_dataset, 
      model,
      optimizer,
      train_acc_metric,
      val_acc_metric,
      epochs=config.epochs, 
      log_step=config.log_step, 
      val_log_step=config.val_log_step)

run.finish()  # In Jupyter/Colab, let us know you're finished!

In [ ]:
# initialize wandb with your project name and optionally with configutations.
# play around with the config values and see the result on your wandb dashboard.
config = {
              "learning_rate": 0.001,
              "epochs": 100,
              "batch_size": 64,
              "log_step": 200,
              "val_log_step": 50,
           }

run = wandb.init(project='dense-model-activations-fixed-seed', config=config,  group='RMSprop_L1e-3', name='Dense_Model_Hard_Sigmoid')
config = wandb.config

# Initialize model.
# make_dense_model_elu()
# make_dense_model_gelu
# make_dense_model_plus
# make_dense_model_hard_sigmoid
# make_dense_model_sigmoid

model = make_dense_model_hard_sigmoid()
model.summary()
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.RMSprop(learning_rate=config.learning_rate)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

train(train_dataset,
      test_dataset, 
      model,
      optimizer,
      train_acc_metric,
      val_acc_metric,
      epochs=config.epochs, 
      log_step=config.log_step, 
      val_log_step=config.val_log_step)

run.finish()  # In Jupyter/Colab, let us know you're finished!

In [ ]:
# initialize wandb with your project name and optionally with configutations.
# play around with the config values and see the result on your wandb dashboard.
config = {
              "learning_rate": 0.001,
              "epochs": 100,
              "batch_size": 64,
              "log_step": 200,
              "val_log_step": 50,
           }

run = wandb.init(project='dense-model-activations-fixed-seed', config=config, group='RMSprop_L1e-3', name='Dense_Model_Sigmoid_Plus')
config = wandb.config

# Initialize model.
# make_dense_model_elu()
# make_dense_model_gelu
# make_dense_model_plus

model = make_dense_model_plus()
model.summary()
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.RMSprop(learning_rate=config.learning_rate)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

train(train_dataset,
      test_dataset, 
      model,
      optimizer,
      train_acc_metric,
      val_acc_metric,
      epochs=config.epochs, 
      log_step=config.log_step, 
      val_log_step=config.val_log_step)

run.finish()  # In Jupyter/Colab, let us know you're finished!

In [ ]:
# initialize wandb with your project name and optionally with configutations.
# play around with the config values and see the result on your wandb dashboard.
config = {
              "learning_rate": 0.001,
              "epochs": 100,
              "batch_size": 64,
              "log_step": 200,
              "val_log_step": 50,
           }

run = wandb.init(project='dense-model-activations-fixed-seed', config=config, group='RMSprop_L1e-3', name='Dense_Model_Gelu')
config = wandb.config

# Initialize model.
# make_dense_model_elu()

model = make_dense_model_gelu()
model.summary()
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.RMSprop(learning_rate=config.learning_rate)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

train(train_dataset,
      test_dataset, 
      model,
      optimizer,
      train_acc_metric,
      val_acc_metric,
      epochs=config.epochs, 
      log_step=config.log_step, 
      val_log_step=config.val_log_step)

run.finish()  # In Jupyter/Colab, let us know you're finished!

In [ ]:
# initialize wandb with your project name and optionally with configutations.
# play around with the config values and see the result on your wandb dashboard.
config = {
              "learning_rate": 0.001,
              "epochs": 100,
              "batch_size": 64,
              "log_step": 200,
              "val_log_step": 50,
           }

run = wandb.init(project='dense-model-activations-fixed-seed', config=config, group='RMSprop_L1e-3', name='Dense_Model_Elu')
config = wandb.config

# Initialize model.
# make_dense_model_elu()

model = make_dense_model_elu()
model.summary()
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.RMSprop(learning_rate=config.learning_rate)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

train(train_dataset,
      test_dataset, 
      model,
      optimizer,
      train_acc_metric,
      val_acc_metric,
      epochs=config.epochs, 
      log_step=config.log_step, 
      val_log_step=config.val_log_step)

run.finish()  # In Jupyter/Colab, let us know you're finished!

In [ ]:
# initialize wandb with your project name and optionally with configutations.
# play around with the config values and see the result on your wandb dashboard.
config = {
              "learning_rate": 0.001,
              "epochs": 100,
              "batch_size": 64,
              "log_step": 200,
              "val_log_step": 50,
           }

run = wandb.init(project='dense-model-activations-fixed-seed', config=config, group='RMSprop_L1e-3', name='Dense_Model_Relu')
config = wandb.config


model = make_dense_model()
model.summary()
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.RMSprop(learning_rate=config.learning_rate)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

train(train_dataset,
      test_dataset, 
      model,
      optimizer,
      train_acc_metric,
      val_acc_metric,
      epochs=config.epochs, 
      log_step=config.log_step, 
      val_log_step=config.val_log_step)

run.finish()  # In Jupyter/Colab, let us know you're finished!

In [ ]:
# initialize wandb with your project name and optionally with configutations.
# play around with the config values and see the result on your wandb dashboard.
config = {
              "learning_rate": 0.001,
              "epochs": 100,
              "batch_size": 64,
              "log_step": 200,
              "val_log_step": 50,
           }

run = wandb.init(project='dense-model-activations-fixed-seed', config=config, group='RMSprop_L1e-3', name='Dense_Model_Swish')
config = wandb.config


model = make_dense_model_swish()
model.summary()
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.RMSprop(learning_rate=config.learning_rate)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

train(train_dataset,
      test_dataset, 
      model,
      optimizer,
      train_acc_metric,
      val_acc_metric,
      epochs=config.epochs, 
      log_step=config.log_step, 
      val_log_step=config.val_log_step)

run.finish()  # In Jupyter/Colab, let us know you're finished!